In [1]:
contig_csv_path = "./motif_specs_with_contigs.csv"
new_contig_csv_path = "./motif_specs_with_contigs_for_motif_files.csv"

In [2]:
f = open(contig_csv_path)
l = f.readline().strip()
f.close()
print(l)

1BCF,A18-25;A47-54;A92-99;A123-130,A19-25;A47-50;A52-53;A92-93;A95-99;A123-126;A128-129,125,8-15;A92-99;16-30;A123-130;16-30;A47-54;16-30;A18-25;8-15


In [3]:
chain_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
def new_contig(rfdiffusion_contig):
    scaffold_segments = {}
    contig_segments = []
    for i, seg in enumerate(rfdiffusion_contig.split(";")):
        if seg[0].isnumeric():
            scaffold_segments[i] = seg
        else:
            chain = seg[0]
            residues = seg[1:]
            if "-" in residues:
                start = int(residues.split("-")[0])
                end = int(residues.split("-")[1])
                new_res_string = f"1-{1+end-start}"
            else:
                start = int(residues)
                new_res_string = "1-1"
            contig_segments.append({"pos_in_contig":i, "res_start":start, "new_res_string":new_res_string})

    sorted_contigs = sorted(contig_segments, key=lambda x: x['res_start'])
    for i, contig_segment in enumerate(sorted_contigs):
        segment_str = chain_labels[i]+contig_segment['new_res_string']
        scaffold_segments[contig_segment['pos_in_contig']] = segment_str

    return ";".join(scaffold_segments[i] for i in range(len(scaffold_segments)))

In [4]:
f_out = open(new_contig_csv_path, 'w')
with open(contig_csv_path) as f:
    for l in f:
        motif_name, redesignable_positions, contig, length, rfdiffusion_contig = l.split(",")
        new_contig_str = new_contig(rfdiffusion_contig.strip())
        new_l = ",".join([motif_name, redesignable_positions, contig, length, new_contig_str])
        f_out.write(new_l + "\n")
f_out.close()